# Vectors

In [9]:
x = Int16[10,5,3,11]

length(x)
Base.unsafe_load(pointer(x))

10

In [17]:
sz = sizeof(eltype(x))
Base.unsafe_load(pointer(x)+3sz)

11

In [21]:
# WARNING: High performance languages are not SAFE
Base.unsafe_load(pointer(x)+20sz) # this may crash Julia

-26620

In [30]:
x = [1.3, 1.2, 1.5]
sz = sizeof(eltype(x))
Base.unsafe_load(pointer(x)+sz) # 8 bytes in 64 bits

1.2

# Matrices

In [33]:
A = [1 2; 
     4 5;
     7 8]

3×2 Matrix{Int64}:
 1  2
 4  5
 7  8

In [39]:
size(A)
sz = sizeof(eltype(A))
Base.unsafe_load(pointer(A) + 3sz)

2

In [40]:
vec(A)

6-element Vector{Int64}:
 1
 4
 7
 2
 5
 8

## Matrix multiplication

In [51]:
function mul_rows(A, x)
    m, n = size(A)
    ret = zeros(m) # assumes Float64
    for k = 1:m
        for j = 1:n
            ret[k] += A[k,j] * x[j]
        end
    end
    ret
end

function mul(A, x) # go down columns, access A in order
    m, n = size(A)
    ret = zeros(m) # assumes Float64
    for j = 1:n
        for k = 1:m
            ret[k] += A[k,j] * x[j]
        end
    end
    ret
end

mul (generic function with 1 method)

In [57]:
n = 100
A = randn(n,n)
x = rand(n);

using BenchmarkTools
@btime mul_rows(A, x);
@btime mul(A, x);

  18.206 μs (1 allocation: 896 bytes)
  6.631 μs (1 allocation: 896 bytes)


In [59]:
@btime A*x;

  1.875 μs (1 allocation: 896 bytes)


In [58]:
mul_rows(A, x) ≈ mul(A, x) ≈ A*x

true

## Triangular matrices

In [72]:
using LinearAlgebra

data = [1 2 3;
        4 5 6;
        7 8 9]
U = UpperTriangular(data) # ignore below diagonal
L = LowerTriangular(data) # ignore above diagonal

b = [6,4,3]
x = U \ b
y = L \ b

3-element Vector{Float64}:
  6.0
 -4.0
 -0.7777777777777778

In [76]:
U*x ≈ b
L*y ≈ b

true

## Diagonal matrices

In [111]:
n = 200
x = randn(n)
A = randn(n,n)
D = Diagonal(randn(n));

In [113]:
@btime D*x;
@btime A*x;

  126.301 ns (1 allocation: 1.77 KiB)
  3.960 μs (1 allocation: 1.77 KiB)


In [115]:
@btime D\x;
@btime A\x;

  148.671 ns (1 allocation: 1.77 KiB)
  232.014 μs (4 allocations: 316.08 KiB)


## Bidiagonal

In [120]:
U = Bidiagonal(randn(n), randn(n-1), :U)
L = Bidiagonal(randn(n), randn(n-1), :L);

@btime U \ x;

  1.115 μs (1 allocation: 1.77 KiB)


## Tridiagonal

In [125]:
T = Tridiagonal(randn(n-1),randn(n), randn(n-1))

@btime T * x

  234.062 ns (1 allocation: 1.77 KiB)


200-element Vector{Float64}:
 -1.8032288411133814
  1.3483582075439555
 -2.4882024715679836
  2.646568887163777
 -4.764836422499411
 -1.133009691303415
 -1.005592672745377
  0.6725978542547606
  1.679039198340986
  0.8926420235242756
 -0.9375286129463626
 -1.356943759968118
  0.3118038613602252
  ⋮
  0.5906182790359616
 -0.6683875913127153
 -2.3189332838835
  0.5224201240780477
  1.10220326349299
  1.825020329154461
  0.8848964486589632
  1.0264158788361584
  0.08669767235818893
 -0.7163637406637045
 -1.1603467712437037
  1.8690881363441245